In [66]:
import re
import pandas as pd

In [102]:
# file = 'LocalSP2002/LocalSP2002capital.TXT'
# regiao = 'capital'

file = 'LocalSP2002/LocalSP2002estado.TXT'
regiao = 'estado'

with open(f'data/{file}', 'r', errors='ignore') as f:
    data = f.readlines()

In [103]:
# removendo linhas em branco
data = list(filter(lambda linha: linha != '\n', data))

In [104]:
# removendo cabecalho
data = list(filter(lambda linha: not linha.startswith(' Tribunal Regional Eleitoral - SP'), data))
data = list(filter(lambda linha: not linha.startswith(' Sistema de Alistamento Eleitoral'), data))
data = list(filter(lambda linha: not linha.startswith(' Locais de Votacao/Secoes (Ordenado por ZONA)'), data))
data = list(filter(lambda linha: not linha.__contains__('---------------------------------------------------'), data))

In [105]:
data

[' Locais de Votacao/Secoes (Ordenado por MUNICIPIO)     Pagina:     001 /  1.768\n',
 ' Municipio: 61018 - ADAMANTINA\n',
 ' Zona: 157\n',
 ' Local     : 1031 - E.E. "HELEN KELLER"\n',
 ' Endereco  : RUA MARIO OLIVERO, N 122     VILA CICMA         000000\n',
 ' Bairro    : CENTRO\n',
 ' Cep       :  17800-000   Qtd. Secoes Possiveis:  0015\n',
 ' Secao/Eleitores:  0018/0533 0019/0532 0020/0532 0021/0532 0022/0535 0023/0533\n',
 ' Total Secoes/Eleitores:  006 /     3.197\n',
 ' Local     : 1929 - E.E. HELEN KELLER\n',
 ' Endereco  : R. MARIO OLIVERO N. 122      ADAMANTINA         000000\n',
 ' Bairro    : CENTRO\n',
 ' Cep       :  17800-000   Qtd. Secoes Possiveis:  0001\n',
 ' Secao/Eleitores:  0068/0340\n',
 ' Total Secoes/Eleitores:  001 /       340\n',
 ' Local     : 1058 - E.E. PROF. TERUYO KIKUTA\n',
 ' Endereco  : RUA DOS JASMINS, 154         VILA JARDIM        000000\n',
 ' Bairro    : CENTRO\n',
 ' Cep       :  17800-000   Qtd. Secoes Possiveis:  0012\n',
 ' Secao/Eleitores: 

In [106]:
# removendo quebra de linha e espaco no comeco
data = list(map(lambda linha: linha[1:-1], data))

In [107]:
data

['Locais de Votacao/Secoes (Ordenado por MUNICIPIO)     Pagina:     001 /  1.768',
 'Municipio: 61018 - ADAMANTINA',
 'Zona: 157',
 'Local     : 1031 - E.E. "HELEN KELLER"',
 'Endereco  : RUA MARIO OLIVERO, N 122     VILA CICMA         000000',
 'Bairro    : CENTRO',
 'Cep       :  17800-000   Qtd. Secoes Possiveis:  0015',
 'Secao/Eleitores:  0018/0533 0019/0532 0020/0532 0021/0532 0022/0535 0023/0533',
 'Total Secoes/Eleitores:  006 /     3.197',
 'Local     : 1929 - E.E. HELEN KELLER',
 'Endereco  : R. MARIO OLIVERO N. 122      ADAMANTINA         000000',
 'Bairro    : CENTRO',
 'Cep       :  17800-000   Qtd. Secoes Possiveis:  0001',
 'Secao/Eleitores:  0068/0340',
 'Total Secoes/Eleitores:  001 /       340',
 'Local     : 1058 - E.E. PROF. TERUYO KIKUTA',
 'Endereco  : RUA DOS JASMINS, 154         VILA JARDIM        000000',
 'Bairro    : CENTRO',
 'Cep       :  17800-000   Qtd. Secoes Possiveis:  0012',
 'Secao/Eleitores:  0029/0536 0030/0533 0031/0536 0032/0535 0033/0535 0034/05

In [108]:
# Iterando sobre cada linha

resultado = []
padrao_cep = re.compile("[0-9]{5}[-]{0,1}[0-9]{3}")
padrao_secao_eleitores = re.compile("[0-9]{4}[/][0-9]+")
valida_secoes, valida_eleitores = [], []

for i in data:
    if i.startswith('     Zona:'):
        zona = i.split(':')[-1].lstrip()

    if i.startswith('Municipio'):
        cod_municipio = i.split(' - ')[0].split(': ')[-1].lstrip()
        municipio = i.split(' - ')[-1]

    if i.startswith('Local'):
        cod_local = i.split(': ')[-1].split(' - ')[0]
        local = i.split(': ')[-1].split(' - ')[1]
    
    if i.startswith('Endereco  :'):
        endereco = i.split(':')[-1].lstrip()

    if i.startswith('Bairro    :'):
        bairro = i.split(':')[-1].lstrip()

    if i.startswith('Cep       :'):
        cep = padrao_cep.search(i).group()

    if i.startswith('Secao/Eleitores:') or i.startswith('             '):
        secao_eleitores = padrao_secao_eleitores.findall(i)
        for s in secao_eleitores:
            secao = s.split('/')[0]
            eleitores = int(s.split('/')[1])
            resultado.append([cod_municipio, municipio, cod_local, local, endereco, bairro, cep, zona, secao, eleitores])
        # break

    if i.startswith('Total Secoes/Eleitores:'):
        valida_secoes.append(int(i.split(' /')[0].split(':')[-1]))
        valida_eleitores.append(int(i.split(' /')[-1].replace('.', '')))
        # resultado.append([cod_municipio, municipio, cod_local, local, endereco, bairro, cep, secao, eleitores])
        # break

    


In [109]:
df_locais_votacao = pd.DataFrame(resultado, columns=['cod_municipio', 'municipio', 'cod_local', 'local', 'endereco', 'bairro', 'cep', 'zona', 'secao', 'eleitores'])

In [110]:
df_locais_votacao

,cod_municipio,municipio,cod_local,local,endereco,bairro,cep,zona,secao,eleitores
0,61018,ADAMANTINA,1031,"E.E. ""HELEN KELLER""","RUA MARIO OLIVERO, N 122 VILA CICMA ...",CENTRO,17800-000,376,0018,533
1,61018,ADAMANTINA,1031,"E.E. ""HELEN KELLER""","RUA MARIO OLIVERO, N 122 VILA CICMA ...",CENTRO,17800-000,376,0019,532
2,61018,ADAMANTINA,1031,"E.E. ""HELEN KELLER""","RUA MARIO OLIVERO, N 122 VILA CICMA ...",CENTRO,17800-000,376,0020,532
3,61018,ADAMANTINA,1031,"E.E. ""HELEN KELLER""","RUA MARIO OLIVERO, N 122 VILA CICMA ...",CENTRO,17800-000,376,0021,532
4,61018,ADAMANTINA,1031,"E.E. ""HELEN KELLER""","RUA MARIO OLIVERO, N 122 VILA CICMA ...",CENTRO,17800-000,376,0022,535
...,...,...,...,...,...,...,...,...,...,...
49611,72451,VOTUPORANGA,1031,SECRETARIA MUNICIPAL DA EDUCACAO,"R.CEARA, 962",DO CAFE,15500-000,376,0030,456
49612,72451,VOTUPORANGA,1031,SECRETARIA MUNICIPAL DA EDUCACAO,"R.CEARA, 962",DO CAFE,15500-000,376,0031,458
49613,61921,ZACARIAS,1023,EEPG ANTONIO TEIXEIRA DOS SANTOS,"RUA DOM PEDRO II ,1000 - DISTRITO DE ZACARIAS",CENTRO,00015-265,376,0021,599
49614,61921,ZACARIAS,1023,EEPG ANTONIO TEIXEIRA DOS SANTOS,"RUA DOM PEDRO II ,1000 - DISTRITO DE ZACARIAS",CENTRO,00015-265,376,0022,598


In [111]:
sum(valida_secoes)

49616

In [112]:
sum(valida_eleitores)

25655553

In [113]:
df_locais_votacao.eleitores.sum()

25655553

In [114]:
df_locais_votacao.to_csv(f'resultados/locais_de_votacao-2002{regiao}.csv')